<a href="https://colab.research.google.com/github/vinayjoshva/PDS-assignments/blob/main/Assignment2_16352584.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np

# Load the CSV file into a DataFrame
df = pd.read_csv('train.csv')

# Display the first few rows of the DataFrame
print(df)


      Unnamed: 0                              Name    Location  Year  \
0              1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1              2                      Honda Jazz V     Chennai  2011   
2              3                 Maruti Ertiga VDI     Chennai  2012   
3              4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4              6            Nissan Micra Diesel XV      Jaipur  2013   
...          ...                               ...         ...   ...   
5842        6014                  Maruti Swift VDI       Delhi  2014   
5843        6015          Hyundai Xcent 1.1 CRDi S      Jaipur  2015   
5844        6016             Mahindra Xylo D4 BSIV      Jaipur  2012   
5845        6017                Maruti Wagon R VXI     Kolkata  2013   
5846        6018             Chevrolet Beat Diesel   Hyderabad  2011   

      Kilometers_Driven Fuel_Type Transmission Owner_Type     Mileage  \
0                 41000    Diesel       Manual      First  19.

In [22]:
rows, columns = df.shape

print(f"Number of rows: {rows}")
print(f"Number of columns: {columns}")

Number of rows: 5847
Number of columns: 14


In [23]:
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)

Missing values per column:
 Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In [33]:
# Clean and convert 'Power' column
df['Power'] = df['Power'].astype(str).str.replace(' bhp', '')  # Convert to string first, then remove ' bhp'
df['Power'] = pd.to_numeric(df['Power'], errors='coerce')       # Convert to float, with errors converted to NaN
df['Power'].fillna(df['Power'].median())          # Impute missing values with the median

# Clean and convert 'Mileage' column
df['Mileage'] = df['Mileage'].astype(str).str.replace(' kmpl', '').str.replace(' km/kg', '')  # Convert to string first, then clean up
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')    # Convert to float, with errors converted to NaN
df['Mileage'].fillna(df['Mileage'].median())       # Impute missing values with the median

# I am imputing the columns with median values because the data is skewed numerical columns also removing the units in the column

,Mileage
0,19.67
1,13.00
2,20.77
3,15.20
4,23.08
...,...
5842,28.40
5843,24.40
5844,14.00
5845,18.90


In [42]:
# Clean and convert 'Engine' column
df['Engine'] = df['Engine'].astype(str).str.replace(' CC', '')
df['Engine'] = pd.to_numeric(df['Engine'], errors='coerce')
df['Engine'].fillna(df['Engine'].median())

#again I am using median as it is more robust and represents
# central value which is not affected by extreme values whereas mean can be misleading

,Engine
0,1582.0
1,1199.0
2,1248.0
3,1968.0
4,1461.0
...,...
5842,1248.0
5843,1120.0
5844,2498.0
5845,998.0


In [48]:
# Fill missing values in 'Seats' with mode
df['Seats'].fillna(df['Seats'].mode()[0])

#I am using mode here because it represents most common values in the column as it is
#ordinal in nature and discrete in count. If we use mean/median it could have resulted in decimal values which makes no sense for a column like SEATS

,Seats
0,5.0
1,5.0
2,7.0
3,5.0
4,5.0
...,...
5842,5.0
5843,5.0
5844,8.0
5845,5.0


In [51]:
# performing one hot encoding on the categorical columns to make analysis better
categorical_cols = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Location']
data = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print(data.head())

   Unnamed: 0                              Name  Year  Kilometers_Driven  \
0           1  Hyundai Creta 1.6 CRDi SX Option  2015              41000   
1           2                      Honda Jazz V  2011              46000   
2           3                 Maruti Ertiga VDI  2012              87000   
3           4   Audi A4 New 2.0 TDI Multitronic  2013              40670   
4           6            Nissan Micra Diesel XV  2013              86999   

   Mileage  Engine   Power  Seats  New_Price  Price  ...  Location_Bangalore  \
0    19.67  1582.0  126.20    5.0      1.175  12.50  ...               False   
1    13.00  1199.0   88.70    5.0      0.861   4.50  ...               False   
2    20.77  1248.0   88.76    7.0      1.175   6.00  ...               False   
3    15.20  1968.0  140.80    5.0      1.175  17.74  ...               False   
4    23.08  1461.0   63.10    5.0      1.175   3.50  ...               False   

   Location_Chennai  Location_Coimbatore  Location_Delhi  Loca

In [52]:
# Verify that all missing values are handled
print("Missing values after cleaning:\n", df.isnull().sum())


Missing values after cleaning:
 Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64


In [56]:
#adding new feature 'Power_to_weight' ratio to check cars with higher power-to-weight ratio tend to performe better which could be important for pricing metrics
data['Power_to_Weight'] = data['Power'] / data['Engine'] * 1000

print(data[['Name', 'Power', 'Engine', 'Power_to_Weight']].head())

                               Name   Power  Engine  Power_to_Weight
0  Hyundai Creta 1.6 CRDi SX Option  126.20  1582.0        79.772440
1                      Honda Jazz V   88.70  1199.0        73.978315
2                 Maruti Ertiga VDI   88.76  1248.0        71.121795
3   Audi A4 New 2.0 TDI Multitronic  140.80  1968.0        71.544715
4            Nissan Micra Diesel XV   63.10  1461.0        43.189596


In [63]:
# Select specific columns
selected_data = data[['Name', 'Year', 'Price']]

# Filter rows where price is above 10
filtered_data = data[data['Price'] > 10]

# Rename columns
renamed_data = data.rename(columns={'Price': 'Selling_Price', 'Kilometers_Driven': 'Mileage'})

# Add a new column for car age
data['Car_Age'] = 2024 - data['Year']

# Sort by price in descending order
arranged_data = data.sort_values(by='Price', ascending=False)

# Summarize data: calculate average and max price by location
summary_data = data.groupby('Price').agg(
    avg_price=('Price', 'mean'),
    max_price=('Price', 'max')
).reset_index()

print(selected_data.head())
print("-----------------------------------------")
print(filtered_data.head())
print("-----------------------------------------")
print(renamed_data.head())
print("-----------------------------------------")
print(arranged_data.head())
print("-----------------------------------------")
print(summary_data.head())

                               Name  Year  Price
0  Hyundai Creta 1.6 CRDi SX Option  2015  12.50
1                      Honda Jazz V  2011   4.50
2                 Maruti Ertiga VDI  2012   6.00
3   Audi A4 New 2.0 TDI Multitronic  2013  17.74
4            Nissan Micra Diesel XV  2013   3.50
-----------------------------------------
    Unnamed: 0                               Name  Year  Kilometers_Driven  \
0            1   Hyundai Creta 1.6 CRDi SX Option  2015              41000   
3            4    Audi A4 New 2.0 TDI Multitronic  2013              40670   
5            7  Toyota Innova Crysta 2.8 GX AT 8S  2016              36000   
11          13   Land Rover Range Rover 2.2L Pure  2014              72000   
12          14     Land Rover Freelander 2 TD4 SE  2012              85000   

    Mileage  Engine  Power  Seats  New_Price  Price  ...  Location_Coimbatore  \
0     19.67  1582.0  126.2    5.0      1.175  12.50  ...                False   
3     15.20  1968.0  140.8    5.0